# enviorment (package)

In [2]:
%matplotlib inline
import csv
import pandas as pd
import numpy as np
import networkx as nx
import mygene

# material process

In [9]:
network_file_dir = r'../../../master_research/data/STRING_network_process_GeneSymbol.txt'
network_file = open(network_file_dir,mode='rb')
network = nx.read_weighted_edgelist(network_file,create_using=nx.DiGraph())
print(nx.info(network))

Name: 
Type: DiGraph
Number of nodes: 18455
Number of edges: 11150694
Average in degree: 604.2099
Average out degree: 604.2099


In [10]:
#physical_network = r'D:/downloads/9606.protein.links.v11.txt/9606.protein.links.v11.txt'
network = r'D:/downloads/9606.protein.links.v11.5.txt/9606.protein.links.v11.5.txt'
data = pd.read_csv(network,sep=" ")
print (data)
print(type(data))

                      protein1              protein2  combined_score
0         9606.ENSP00000000233  9606.ENSP00000379496             155
1         9606.ENSP00000000233  9606.ENSP00000314067             197
2         9606.ENSP00000000233  9606.ENSP00000263116             222
3         9606.ENSP00000000233  9606.ENSP00000361263             181
4         9606.ENSP00000000233  9606.ENSP00000409666             270
...                        ...                   ...             ...
11938493  9606.ENSP00000485678  9606.ENSP00000354800             213
11938494  9606.ENSP00000485678  9606.ENSP00000308270             151
11938495  9606.ENSP00000485678  9606.ENSP00000335660             181
11938496  9606.ENSP00000485678  9606.ENSP00000300127             154
11938497  9606.ENSP00000485678  9606.ENSP00000310488             167

[11938498 rows x 3 columns]
<class 'pandas.core.frame.DataFrame'>


In [11]:
data["protein1"] = data['protein1'].str.lstrip("9609.")

data["protein2"] = data['protein2'].str.lstrip("9609.")

data['combined_score'] = data["combined_score"]/1000


In [13]:
type(data['combined_score'][1])
data.to_csv(r'../../data/STRINGv11Cut9606.txt', header=['#protein1','protein2','combined_score'], index=None, sep=' ', mode='w')
#如果沒有在protein1前面加上#的話，會讓下一步的讀檔出問題


In [14]:
#check dataframe to graph，如果設定graph()的話，好像會吃掉一些edge
#所以要設定成digraph
#但不知道為什麼，沒有辦法加上權重
#透過先寫檔再讀檔的方式來解決問題

network_file = open(r'../../data/STRINGv11Cut9606.txt','rb')
G = nx.read_weighted_edgelist(network_file,comments='#',create_using=nx.DiGraph())

#G = nx.from_pandas_edgelist(data,'protein1','protein2',edge_attr='combined_score',create_using=nx.DiGraph()) #一直沒辦法加入權重的寫法

In [15]:
print(nx.info(G))

Name: 
Type: DiGraph
Number of nodes: 19385
Number of edges: 11938498
Average in degree: 615.8627
Average out degree: 615.8627


In [16]:
print(nx.is_weighted(G))

True


In [17]:
print(nx.is_weighted(G,('ENSP00000000233','ENSP00000379496')))

True


In [18]:
peptideID = data["protein1"].append(data["protein2"]).unique()

In [19]:
print(len(peptideID))
peptideID

19385


array(['ENSP00000000233', 'ENSP00000000412', 'ENSP00000001008', ...,
       'ENSP00000485668', 'ENSP00000485672', 'ENSP00000485678'],
      dtype=object)

## high  confidence? to nature paper ?300w edge?

# tranfer peptideID to geneID

In [20]:
mg = mygene.MyGeneInfo()
results_string_physical= mg.querymany(peptideID, scopes='ensembl.protein',fields = 'symbol',species='human',returnall=True,as_dataframe=True,df_index=True,verbose=True)
#19000個輸入，然後有608個找不到transfer gene id，但是這邊的數字，怎麼會比high confidence score的部分還要少...
#del peptideID

querying 1-1000...done.
querying 1001-2000...done.
querying 2001-3000...done.
querying 3001-4000...done.
querying 4001-5000...done.
querying 5001-6000...done.
querying 6001-7000...done.
querying 7001-8000...done.
querying 8001-9000...done.
querying 9001-10000...done.
querying 10001-11000...done.
querying 11001-12000...done.
querying 12001-13000...done.
querying 13001-14000...done.
querying 14001-15000...done.
querying 15001-16000...done.
querying 16001-17000...done.
querying 17001-18000...done.
querying 18001-19000...done.
querying 19001-19385...done.
Finished.
767 input query terms found no hit:
	['ENSP00000035383', 'ENSP00000062104', 'ENSP00000195455', 'ENSP00000204615', 'ENSP00000205890', 'ENS


In [21]:
print(results_string_physical['out']['symbol'])

query
ENSP00000000233       ARF5
ENSP00000000412       M6PR
ENSP00000001008      FKBP4
ENSP00000001146    CYP26B1
ENSP00000002125    NDUFAF7
                    ...   
ENSP00000485663      EIF3L
ENSP00000485664        NaN
ENSP00000485668     OR4N4C
ENSP00000485672        NaN
ENSP00000485678        NaN
Name: symbol, Length: 19385, dtype: object


In [22]:
print(results_string_physical)

{'out':                              _id     _score   symbol notfound
query                                                        
ENSP00000000233              381  27.463379     ARF5      NaN
ENSP00000000412             4074  27.463379     M6PR      NaN
ENSP00000001008             2288  27.011208    FKBP4      NaN
ENSP00000001146            56603  27.463379  CYP26B1      NaN
ENSP00000002125            55471  27.463379  NDUFAF7      NaN
...                          ...        ...      ...      ...
ENSP00000485663            51386  27.463379    EIF3L      NaN
ENSP00000485664  ENSG00000279493  28.987856      NaN      NaN
ENSP00000485668  ENSG00000279408  28.987856   OR4N4C      NaN
ENSP00000485672              NaN        NaN      NaN     True
ENSP00000485678              NaN        NaN      NaN     True

[19385 rows x 4 columns], 'dup': Empty DataFrame
Columns: [query, duplicate hits]
Index: [], 'missing':                query
0    ENSP00000035383
1    ENSP00000062104
2    ENSP000001954

In [23]:
string_physical_IDTransfer = results_string_physical['out'].reset_index()

In [24]:
#string_physical_IDTransfer = string_physical_IDTransfer.rename(mapper={'ensembl.gene':'ensembl_gene'},axis=1)

In [25]:
#get whole ensembl gene id 
string_physical_gene_id = string_physical_IDTransfer['symbol'].unique()
print(len(string_physical_gene_id))

18421


In [26]:
string_physical_ensembl_gene_list = string_physical_IDTransfer[['query','symbol']]
print(string_physical_ensembl_gene_list)

                 query   symbol
0      ENSP00000000233     ARF5
1      ENSP00000000412     M6PR
2      ENSP00000001008    FKBP4
3      ENSP00000001146  CYP26B1
4      ENSP00000002125  NDUFAF7
...                ...      ...
19380  ENSP00000485663    EIF3L
19381  ENSP00000485664      NaN
19382  ENSP00000485668   OR4N4C
19383  ENSP00000485672      NaN
19384  ENSP00000485678      NaN

[19385 rows x 2 columns]


In [27]:
#relabel node
print(type(results_string_physical['missing']['query']))
print(results_string_physical['missing'])

<class 'pandas.core.series.Series'>
               query
0    ENSP00000035383
1    ENSP00000062104
2    ENSP00000195455
3    ENSP00000204615
4    ENSP00000205890
..               ...
762  ENSP00000485589
763  ENSP00000485610
764  ENSP00000485629
765  ENSP00000485672
766  ENSP00000485678

[767 rows x 1 columns]


In [28]:
string_physical_network = G.copy()
#string_physical_ensembl_gene_list = string_physical_ensembl_gene_list.query('ensembl_gene' != 'NaN')
print(nx.info(string_physical_network))

Name: 
Type: DiGraph
Number of nodes: 19385
Number of edges: 11938498
Average in degree: 615.8627
Average out degree: 615.8627


In [29]:
#for i in range(len(results_string_physical['missing'])):
    #print(results_string_physical['missing'].iloc[i])
missing = results_string_physical['missing']['query'].tolist()
for i in missing:
    string_physical_network.remove_node(i)
print(nx.info(string_physical_network))

Name: 
Type: DiGraph
Number of nodes: 18618
Number of edges: 11273084
Average in degree: 605.4938
Average out degree: 605.4938


In [30]:
print(nx.is_weighted(string_physical_network))

True


# rename the dataframe

先建構成網路，然後再把前面對應到的字詞抽出來，然後mapping上去

In [31]:
print(string_physical_ensembl_gene_list)

                 query   symbol
0      ENSP00000000233     ARF5
1      ENSP00000000412     M6PR
2      ENSP00000001008    FKBP4
3      ENSP00000001146  CYP26B1
4      ENSP00000002125  NDUFAF7
...                ...      ...
19380  ENSP00000485663    EIF3L
19381  ENSP00000485664      NaN
19382  ENSP00000485668   OR4N4C
19383  ENSP00000485672      NaN
19384  ENSP00000485678      NaN

[19385 rows x 2 columns]


In [32]:
string_physical_ensembl_gene_list = string_physical_ensembl_gene_list[['query','symbol']]
print(string_physical_ensembl_gene_list)

                 query   symbol
0      ENSP00000000233     ARF5
1      ENSP00000000412     M6PR
2      ENSP00000001008    FKBP4
3      ENSP00000001146  CYP26B1
4      ENSP00000002125  NDUFAF7
...                ...      ...
19380  ENSP00000485663    EIF3L
19381  ENSP00000485664      NaN
19382  ENSP00000485668   OR4N4C
19383  ENSP00000485672      NaN
19384  ENSP00000485678      NaN

[19385 rows x 2 columns]


In [33]:
string_physical_ensembl_gene_list['symbol'] = string_physical_ensembl_gene_list['symbol'].map(np.nan_to_num)
print(string_physical_ensembl_gene_list)

                 query   symbol
0      ENSP00000000233     ARF5
1      ENSP00000000412     M6PR
2      ENSP00000001008    FKBP4
3      ENSP00000001146  CYP26B1
4      ENSP00000002125  NDUFAF7
...                ...      ...
19380  ENSP00000485663    EIF3L
19381  ENSP00000485664      0.0
19382  ENSP00000485668   OR4N4C
19383  ENSP00000485672      0.0
19384  ENSP00000485678      0.0

[19385 rows x 2 columns]


In [34]:
i = string_physical_ensembl_gene_list[string_physical_ensembl_gene_list.symbol == 0 ].index
print(len(i))

920


In [35]:
string_physical_ensembl_gene_list = string_physical_ensembl_gene_list.drop(i)
print(string_physical_ensembl_gene_list)

                 query    symbol
0      ENSP00000000233      ARF5
1      ENSP00000000412      M6PR
2      ENSP00000001008     FKBP4
3      ENSP00000001146   CYP26B1
4      ENSP00000002125   NDUFAF7
...                ...       ...
19377  ENSP00000485638   PPIAL4F
19378  ENSP00000485656  PRAMEF26
19379  ENSP00000485659    MUC5AC
19380  ENSP00000485663     EIF3L
19382  ENSP00000485668    OR4N4C

[18465 rows x 2 columns]


In [36]:
string_physical_ensembl_gene_dict = string_physical_ensembl_gene_list.set_index('query').T.to_dict('record')
print(string_physical_ensembl_gene_dict)

C:\Users\steve\Anaconda3\lib\site-packages\pandas\core\frame.py:1549: FutureWarning: Using short name for 'orient' is deprecated. Only the options: ('dict', list, 'series', 'split', 'records', 'index') will be used in a future version. Use one of the above to silence this warning.
  warnings.warn(


[{'ENSP00000000233': 'ARF5', 'ENSP00000000412': 'M6PR', 'ENSP00000001008': 'FKBP4', 'ENSP00000001146': 'CYP26B1', 'ENSP00000002125': 'NDUFAF7', 'ENSP00000002165': 'FUCA2', 'ENSP00000002596': 'HS3ST1', 'ENSP00000002829': 'SEMA3F', 'ENSP00000003084': 'CFTR', 'ENSP00000003100': 'CYP51A1', 'ENSP00000003302': 'USP28', 'ENSP00000004531': 'SLC7A2', 'ENSP00000005178': 'PDK4', 'ENSP00000005226': 'USH1C', 'ENSP00000005257': 'RALA', 'ENSP00000005260': 'BAIAP2L1', 'ENSP00000005284': 'CACNG3', 'ENSP00000005286': 'TMEM132A', 'ENSP00000005340': 'DVL2', 'ENSP00000005386': 'RPAP3', 'ENSP00000005587': 'SKAP2', 'ENSP00000005995': 'PRSS21', 'ENSP00000006015': 'HOXA11', 'ENSP00000006053': 'CX3CL1', 'ENSP00000006275': 'TRAPPC6A', 'ENSP00000006526': 'WDR54', 'ENSP00000006658': 'SPATA20', 'ENSP00000006724': 'CEACAM7', 'ENSP00000006777': 'RHBDD2', 'ENSP00000007390': 'TSR3', 'ENSP00000007414': 'OSBPL7', 'ENSP00000007699': 'YBX2', 'ENSP00000007722': 'ITGA3', 'ENSP00000007735': 'KRT33A', 'ENSP00000008391': 'TFAP2

In [37]:
print(type(string_physical_ensembl_gene_dict))
string_physical_ensembl_gene_dict = string_physical_ensembl_gene_dict[0]
print(type(string_physical_ensembl_gene_dict))

<class 'list'>
<class 'dict'>


In [39]:
print(type(string_physical_network))
string_physical_network = nx.relabel_nodes(string_physical_network,string_physical_ensembl_gene_dict)

<class 'networkx.classes.digraph.DiGraph'>


In [40]:
###需要修改#####
node = list(nx.nodes(string_physical_network))
a = node.copy()
for i in a:
    if i.startswith("ENSP"):
        string_physical_network.remove_node(i)

In [41]:
print(nx.info(string_physical_network))
string_physical_network_node_list= string_physical_network.nodes()
print(len(string_physical_network_node_list))

Name: 
Type: DiGraph
Number of nodes: 18420
Number of edges: 11124445
Average in degree: 603.9330
Average out degree: 603.9330
18420


In [43]:
#write STRING-geneID network file
nx.readwrite.edgelist.write_weighted_edgelist(G=string_physical_network,path='../../data/STRING11_network_process_GeneSymbol.txt',delimiter='\t')

# Construct Network

In [63]:
print(nx.info(string_physical_network))

Name: 
Type: DiGraph
Number of nodes: 17351
Number of edges: 7238712
Average in degree: 417.1928
Average out degree: 417.1928


In [64]:
string_physical_network.remove_nodes_from(list(nx.isolates(string_physical_network)))
print(nx.info(string_physical_network))

Name: 
Type: DiGraph
Number of nodes: 17351
Number of edges: 7238712
Average in degree: 417.1928
Average out degree: 417.1928


In [58]:
string_physical_network_undirected = string_physical_network.to_undirected()
largest_cc_test = max(nx.connected_components(string_physical_network_undirected), key=len)
print(len(largest_cc_test))

17351


In [60]:
node_not_in_large_cc = []
for i in node:
    if i not in largest_cc_test:
        node_not_in_large_cc.append(i)
for k in node:
    if k in node_not_in_large_cc:
        string_physical_network.remove_node(k)
print(nx.info(network_test))

NetworkXError: The node ENSP00000347689 is not in the digraph.

In [61]:
network_test = string_physical_network.copy()
for (u, v, wt) in string_physical_network.edges.data('weight'):
    if wt < 0.7:
        network_test.remove_edge(u, v)
print(nx.info(network_test))

Name: 
Type: DiGraph
Number of nodes: 17351
Number of edges: 508930
Average in degree:  29.3315
Average out degree:  29.3315


In [62]:
network_test.remove_nodes_from(list(nx.isolates(network_test)))
print(nx.info(network_test))

Name: 
Type: DiGraph
Number of nodes: 14147
Number of edges: 508930
Average in degree:  35.9744
Average out degree:  35.9744
